In [ ]:
### 描述

go协程之间用通道通信，如何将它改造成事件形式。


### 需求描述

以下是我的需求

```go
e := NewEvent()
//协程1等待结果, 10是等待时间
e.Wait(10)
//协程2发送结果
e.Send(xxxx)
```

### 原理

原理比较简单，不描述了

### 示例

```go
package common

import (
	"go.uber.org/zap"
	"sync"
	"time"
)

var NOT_USED interface{} = struct{}{}

func NewEvent() *Event {
	e := &Event{
		waiters: make(map[chan interface{}]bool),
		waitersLock: new(sync.Mutex),
		log: zap.S(),
	}
	e.Reset()
	return e
}

type Event struct{
	waiters map[chan interface{}]bool
	waitersLock *sync.Mutex
	log *zap.SugaredLogger
}

//等待结果
func (e *Event) Wait(timeout time.Duration) interface{}{
    //计时器
	ticker := time.NewTicker(time.Second*timeout)
	defer ticker.Stop()

    //结果通道
	e.waitersLock.Lock()
	resultChan := make(chan interface{}, 1)
	e.waiters[resultChan] = true
	e.waitersLock.Unlock()

    //等待结果
	select{
	case result := <- resultChan:
        //特殊标记用于重置中断
		if result == NOT_USED{
			return nil
		}
		return result
	case <- ticker.C:
		close(resultChan)
	}
    //移除(重复delete也没关系)
    delete(e.waiters, resultChan)
	return nil
}

//发送结果
func (e *Event) Send(result interface{}){
	e.waitersLock.Lock()
	defer e.waitersLock.Unlock()

    //通知每个waiter
	removedChans := make([]chan interface{}, 0)
	for resultChan, _:= range e.waiters{
		select{
		case resultChan <- result:
			removedChans = append(removedChans, resultChan)
		default:
			e.log.Warnf("Event.Send %p resultChan=%d, result=%v", e, len(resultChan), result)
		}
	}
    //通知完移除
	for _, removedChan := range removedChans{
		delete(e.waiters, removedChan)
		close(removedChan)
	}
}

//重置
func (e *Event) Reset(){
	e.waitersLock.Lock()
	defer e.waitersLock.Unlock()

    //重置
	for resultChan, _:= range e.waiters{
        //发送特殊比较，中断waiter
		resultChan <- NOT_USED
		close(resultChan)
	}
    //清空waiter
	e.waiters = make(map[chan interface{}]bool)
}
```

[佛說大乘無量壽莊嚴清淨平等覺經pdf](http://doc.sxjy360.top/book/佛說大乘無量壽莊嚴清淨平等覺經(難字注音).pdf)
[净土大经科注2014-doc](http://doc.sxjy360.top/book/净土大经科注2014-doc.zip)